<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/Transformer/5-3-SQL_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Note: You can access the `data` and `util` subdirectories used in the course. In Jupyter version 6, this is via the File>Open menu. In Jupyter version 7 this is in View> File Browser

> Also note that as models and systems change, the output of the models may shift from the video content.

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1kk91GU9DYG4u8HvwEqdyHVD1FgdPOq4D' -O requirements.txt

In [ ]:
!pip install -r requirements.txt

In [3]:
import lamini

In [5]:
def get_schema():
    return """\
0|Team|TEXT
1|NAME|TEXT
2|Jersey|TEXT
3|POS|TEXT
4|AGE|INT
5|HT|TEXT
6|WT|TEXT
7|COLLEGE|TEXT
8|SALARY|TEXT eg.
"""

In [6]:
# Meta Llama 3 Instruct uses a prompt template, with special tags used to indicate the user query and system prompt.
# You can find the documentation on this [model card](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/#meta-llama-3-instruct).
def make_llama_3_prompt(user, system=""):
    system_prompt = ""
    if system != "":
        system_prompt = (
            f"<|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|>"
        )
    return f"<|begin_of_text|>{system_prompt}<|start_header_id|>user<|end_header_id|>\n\n{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [7]:
import logging


def setup_logging():
    # Remove all handlers associated with the root logger object.
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    logging.basicConfig(
        level=logging.WARNING,
        format="%(asctime)s [%(levelname)s] %(message)s",
        handlers=[logging.StreamHandler()],
    )


In [ ]:
!wget https://github.com/meta-llama/llama-recipes/raw/main/recipes/3p_integrations/lamini/text2sql_memory_tuning/nba_roster.db

In [27]:
import sqlite3
import pandas as pd

logger = logging.getLogger(__name__)
engine = sqlite3.connect("./nba_roster.db")
setup_logging()

In [10]:
from google.colab import userdata
lamini.api_key=userdata.get('LAMINI_API_KEY')

In [11]:
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct", api_key = lamini.api_key )

In [28]:
user = """Who is the highest paid NBA player?"""

In [29]:
system = f"""You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
{get_schema()}

Write a sqlite query to answer the following question. Follow instructions exactly"""

In [30]:
print(system)

You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
0|Team|TEXT 
1|NAME|TEXT  
2|Jersey|TEXT 
3|POS|TEXT
4|AGE|INT 
5|HT|TEXT 
6|WT|TEXT 
7|COLLEGE|TEXT 
8|SALARY|TEXT eg. 


Write a sqlite query to answer the following question. Follow instructions exactly


In [31]:
prompt = make_llama_3_prompt(user, system)

In [32]:
print(llm.generate(prompt, max_new_tokens=200))

To answer this question, we can use the following SQLite query:

```sql
SELECT NAME, SALARY
FROM nba_roster
ORDER BY SALARY DESC
LIMIT 1;
```

This query will return the name and salary of the highest paid NBA player. The `ORDER BY SALARY DESC` clause sorts the players by their salary in descending order (highest to lowest), and the `LIMIT 1` clause ensures that we only get the top result.


In [33]:
def get_updated_schema():
    return """\
0|Team|TEXT eg. "Toronto Raptors"
1|NAME|TEXT eg. "Otto Porter Jr."
2|Jersey|TEXT eg. "0" and when null has a value "NA"
3|POS|TEXT eg. "PF"
4|AGE|INT eg. "22" in years
5|HT|TEXT eg. `6' 7"` or `6' 10"`
6|WT|TEXT eg. "232 lbs"
7|COLLEGE|TEXT eg. "Michigan" and when null has a value "--"
8|SALARY|TEXT eg. "$9,945,830" and when null has a value "--"
"""

In [34]:
system = f"""You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
{get_updated_schema()}

Write a sqlite query to answer the following question. Follow instructions exactly"""

In [35]:
prompt = make_llama_3_prompt(user, system)

In [36]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
0|Team|TEXT eg. "Toronto Raptors"
1|NAME|TEXT eg. "Otto Porter Jr."
2|Jersey|TEXT eg. "0" and when null has a value "NA"
3|POS|TEXT eg. "PF"
4|AGE|INT eg. "22" in years
5|HT|TEXT eg. `6' 7"` or `6' 10"`
6|WT|TEXT eg. "232 lbs" 
7|COLLEGE|TEXT eg. "Michigan" and when null has a value "--"
8|SALARY|TEXT eg. "$9,945,830" and when null has a value "--"


Write a sqlite query to answer the following question. Follow instructions exactly<|eot_id|><|start_header_id|>user<|end_header_id|>

Who is the highest paid NBA player?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [37]:
print(llm.generate(prompt, max_new_tokens=200))

To answer this question, we can use the following SQL query:

```sql
SELECT NAME, SALARY
FROM nba_roster
WHERE SALARY!= '--'
ORDER BY CAST(SALARY AS REAL) DESC
LIMIT 1;
```

This query first filters out the rows where the salary is '--' (i.e., the players who don't have a salary listed). Then, it orders the remaining rows by the salary in descending order (highest to lowest). Finally, it returns the top row, which corresponds to the highest paid NBA player.


## Structured Output

We'd like to be able to get just SQL output so we don't have to parse the query from the model response. For this we can use structured output.

In [38]:
result = llm.generate(prompt, output_type={"sqlite_query": "str"}, max_new_tokens=200)

In [39]:
result

{'sqlite_query': "SELECT NAME, SALARY FROM nba_roster WHERE SALARY!= '--' ORDER BY CAST(SALARY AS REAL) DESC LIMIT 1"}

This is great, now we can directly query with the output

In [40]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

# Create a connection to the SQLite database. If the file doesn't exist it will be created.
conn = sqlite3.connect('nba_roster.db')

# Create a cursor object to execute SQL commands.
cursor = conn.cursor()

# Create the nba_roster table if it doesn't exist.
# Assuming the table has columns for 'NAME' and 'SALARY'
# Please modify the column names and types if they differ in your data.
cursor.execute('''
CREATE TABLE IF NOT EXISTS nba_roster (
    NAME TEXT,
    SALARY TEXT
)
''')

# Commit the changes to the database.
conn.commit()

# Now, proceed with your data loading or other operations.
engine = create_engine('sqlite:///nba_roster.db')
df = pd.read_sql(result['sqlite_query'], con=engine)

In [ ]:
# df = pd.read_sql(result['sqlite_query'], con=engine)

In [41]:
df

,NAME,SALARY
0,Saddiq Bey,"$4,556,983"


## Diagnose Hallucinations

The **wrong** query looks like this:

```sql
SELECT NAME, SALARY
FROM nba_roster
WHERE salary != '--'
ORDER BY CAST(SALARY AS REAL) DESC
LIMIT 1;
```


The **correct** query is:

```sql
SELECT salary, name
FROM nba_roster
WHERE salary != '--'
ORDER BY CAST(REPLACE(REPLACE(salary, '$', ''), ',','') AS INTEGER) DESC
LIMIT 1;
```

In [42]:
query="""SELECT salary, name
FROM nba_roster
WHERE salary != '--'
ORDER BY CAST(REPLACE(REPLACE(salary, '$', ''), ',','') AS INTEGER) DESC
LIMIT 1;"""
df = pd.read_sql(query, con=engine)
print(df)

        SALARY           NAME
0  $51,915,615  Stephen Curry
